In [1]:
!unzip proofs.zip

Archive:  proofs.zip
   creating: proofs/
   creating: proofs/3/
   creating: proofs/3/AI-MO_Kimina-Prover-Preview-Distill-7B/
  inflating: proofs/3/AI-MO_Kimina-Prover-Preview-Distill-7B/8.txt  
  inflating: proofs/3/AI-MO_Kimina-Prover-Preview-Distill-7B/3.txt  
  inflating: proofs/3/AI-MO_Kimina-Prover-Preview-Distill-7B/2.txt  
  inflating: proofs/3/AI-MO_Kimina-Prover-Preview-Distill-7B/1.txt  
  inflating: proofs/3/AI-MO_Kimina-Prover-Preview-Distill-7B/5.txt  
  inflating: proofs/3/AI-MO_Kimina-Prover-Preview-Distill-7B/4.txt  
  inflating: proofs/3/AI-MO_Kimina-Prover-Preview-Distill-7B/6.txt  
   creating: proofs/3/AI-MO_Kimina-Prover-Preview-Distill-7B/.ipynb_checkpoints/
  inflating: proofs/3/AI-MO_Kimina-Prover-Preview-Distill-7B/.ipynb_checkpoints/4-checkpoint.txt  
  inflating: proofs/3/AI-MO_Kimina-Prover-Preview-Distill-7B/.ipynb_checkpoints/8-checkpoint.txt  
  inflating: proofs/3/AI-MO_Kimina-Prover-Preview-Distill-7B/.ipynb_checkpoints/5-checkpoint.txt  
  inflating:

  inflating: proofs/2/ByteDance-Seed_BFS-Prover/3.txt  
  inflating: proofs/2/ByteDance-Seed_BFS-Prover/2.txt  
  inflating: proofs/2/ByteDance-Seed_BFS-Prover/1.txt  
  inflating: proofs/2/ByteDance-Seed_BFS-Prover/5.txt  
  inflating: proofs/2/ByteDance-Seed_BFS-Prover/4.txt  
  inflating: proofs/2/ByteDance-Seed_BFS-Prover/6.txt  
  inflating: proofs/2/ByteDance-Seed_BFS-Prover/7.txt  
   creating: proofs/2/Goedel-LM_Goedel-Prover-SFT/
  inflating: proofs/2/Goedel-LM_Goedel-Prover-SFT/8.txt  
  inflating: proofs/2/Goedel-LM_Goedel-Prover-SFT/3.txt  
  inflating: proofs/2/Goedel-LM_Goedel-Prover-SFT/2.txt  
  inflating: proofs/2/Goedel-LM_Goedel-Prover-SFT/1.txt  
  inflating: proofs/2/Goedel-LM_Goedel-Prover-SFT/5.txt  
  inflating: proofs/2/Goedel-LM_Goedel-Prover-SFT/4.txt  
  inflating: proofs/2/Goedel-LM_Goedel-Prover-SFT/6.txt  
  inflating: proofs/2/Goedel-LM_Goedel-Prover-SFT/7.txt  
   creating: proofs/16/
   creating: proofs/16/AI-MO_Kimina-Prover-Preview-Distill-7B/
  infl

In [2]:
# Step 1: Install elan (Lean toolchain manager)
!curl https://raw.githubusercontent.com/leanprover/elan/master/elan-init.sh -sSf | sh -s -- -y

# Step 2: Update Python process PATH so subprocess.run() can find `lean`
import os
elan_bin_path = os.path.expanduser("~/.elan/bin")
os.environ["PATH"] = elan_bin_path + ":" + os.environ["PATH"]

# Verify the installation by checking the version
!lean --version

import os
import subprocess

def setup_lean_project(project_dir="/tmp/lean_project"):
    """
    Creates a Lean project, configures it to use Mathlib,
    and downloads pre-compiled library files.
    """
    print(f"--- Setting up Lean project in: {project_dir} ---")
    os.makedirs(project_dir, exist_ok=True)

    # Content for the lakefile.lean
    lakefile_content = """
    import Lake
    open Lake DSL

    package «lean_project»

    require mathlib from git
      "https://github.com/leanprover-community/mathlib4.git"

    @[default_target]
    lean_lib «lean_project»
    """
    # Write the lakefile
    with open(os.path.join(project_dir, "lakefile.lean"), "w") as f:
        f.write(lakefile_content)

    # Run `lake exe cache get` to download Mathlib's pre-compiled files
    # This is much faster than building from source.
    print("--- Downloading Mathlib cache (this may take a few minutes)... ---")
    try:
        subprocess.run(
            ["lake", "exe", "cache", "get"],
            cwd=project_dir,
            check=True,
            capture_output=True,
            text=True
        )
        print("--- Mathlib cache downloaded successfully. ---")
    except subprocess.CalledProcessError as e:
        print("❌ Error setting up Mathlib cache.")
        print(f"--- STDOUT ---\n{e.stdout}")
        print(f"--- STDERR ---\n{e.stderr}")
        raise  # Stop execution if setup fails

    return project_dir

# --- Call this function once at the start of your script ---
lean_project_path = setup_lean_project()

info: downloading installer
info: default toolchain set to 'stable'
info: downloading https://releases.lean-lang.org/lean4/v4.22.0/lean-4.22.0-linux.tar.zst
376.1 MiB / 376.1 MiB (100 %) 287.8 MiB/s ETA:   0 s
info: installing /home/codespace/.elan/toolchains/leanprover--lean4---v4.22.0
Lean (version 4.22.0, x86_64-unknown-linux-gnu, commit ba2cbbf09d4978f416e0ebd1fceeebc2c4138c05, Release)
--- Setting up Lean project in: /tmp/lean_project ---
--- Downloading Mathlib cache (this may take a few minutes)... ---
--- Mathlib cache downloaded successfully. ---


In [3]:
lean_project_path

'/tmp/lean_project'

In [ ]:
import subprocess
import tempfile
import os
from multiprocessing import Pool
import random
import re

def clean_markdown_fences(code: str) -> str:
    """
    Delete lines that start with one‒three backticks (optionally followed by 'lean').
    Leaves all other back-quotes intact.
    """
    fence_pattern = r'^\s*```.*$'       # lines like ``` or ```lean
    return re.sub(fence_pattern, '', code, flags=re.MULTILINE)


def check_lean_proof(proof_and_context: dict, name: str = 'temp_proof') -> bool:
    """
    Checks a Lean 4 proof string within a configured Lean project using lake.
    """
    proof_string = proof_and_context['proof']
    statement = proof_and_context['formal_statement']
    project_dir = proof_and_context['project_dir']

    # proof_string = re.sub(r'^\s*:=[ \t]+by\b\s*', '', proof_string, count=1)

    full_code = clean_markdown_fences(f"{statement}\n{proof_string}")

    # The file must have a .lean extension
    temp_filename = f"{name}.lean"
    file_path = os.path.join(project_dir, temp_filename)

    try:
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(full_code)

        # --- KEY CHANGE: Use 'lake env lean' to run the compiler ---
        command = ["lake", "env", "lean", temp_filename]

        result = subprocess.run(
            command,
            cwd=project_dir,
            capture_output=True,
            text=True,
            timeout=60
        )

        if result.returncode == 0 and "error:" not in result.stdout:
            # print(f"✅ Proof is correct!\n--- Full Code ---\n{full_code}\n**DONE**")
            return True
        else:
            output = ""
            if result.stdout:
                output += f"\n--- Lean Output (stdout) ---\n{result.stdout}"
            if result.stderr:
                output += f"\n--- Lean Error Output (stderr) ---\n{result.stderr}"
            # print(output)
            # ocasionally print out the full proof full_code
            # if random.random() <= 1/64:
              # print(f"❌ Proof is incorrect or contains errors.\n{output}\n--- Full Code ---\n{full_code}\n**DONE**")
            # print(f"❌ Proof is incorrect or contains errors.\n{output}\n**DONE**")
            return False

    except Exception as e:
        print(f"An error occurred during proof checking: {e}")
        return False

from concurrent.futures import ProcessPoolExecutor, as_completed

def check_proofs_in_parallel(proof_contexts: list[dict], parallel_workers: int = None) -> bool:
    """
    Runs multiple proof checks in parallel, numbered 1..len(proof_contexts).
    Returns True on first success.
    """
    if not proof_contexts:
        return False

    with ProcessPoolExecutor(max_workers=parallel_workers) as executor:
        futures = {
            executor.submit(check_lean_proof, ctx, idx): idx
            for idx, ctx in enumerate(proof_contexts, start=1)
        }
        for future in as_completed(futures):
            try:
                if future.result():
                    return True  # early exit on first successful attempt
            except Exception:
                pass  # optionally log
    return False


In [5]:
# 1. Define the proof and context with the Mathlib header
correct_proof_dict = {
    'formal_statement': 'import Mathlib.Tactic\ntheorem two_plus_two_is_four : 2 + 2 = 4',
    'proof': ':= by rfl',
    'project_dir': lean_project_path
}


check_lean_proof(correct_proof_dict)


True

# Assumptions

In [6]:
# assert that the folder structure is as expected
import os
from typing import Tuple

def check_structure(path: str):
    def check_numeric_children_consecutive(path: str, require_start_zero: bool = True) -> Tuple[int, int]:
        """
        Validate that:
        * `path` exists and is a directory with at least one child.
        * Immediate children are directories named in canonical natural-number form:
            "0", "1", "2", ... (no leading zeros except "0").
        * Their integer values form a consecutive range from min to max.
        * If require_start_zero is True, the range must start at 0.

        Returns:
            (min_value, max_value)

        Raises:
            AssertionError on any violation.
        """
        if not os.path.isdir(path):
            raise AssertionError(f"{path!r} is not a directory")
        entries = os.listdir(path)
        assert entries, f"{path!r} is empty"

        nat_canonical = re.compile(r"0|[1-9][0-9]*\Z")
        nums = []
        for name in entries:
            full = os.path.join(path, name)
            assert os.path.isdir(full), f"{full!r} is not a directory"
            assert nat_canonical.fullmatch(name), (
                f"{name!r} is not a canonical natural number ('0', '1', '2', ... without leading zeros)"
            )
            nums.append(int(name))

        min_n, max_n = min(nums), max(nums)
        if require_start_zero:
            assert min_n == 0, f"Sequence must start at 0 but starts at {min_n}"
        expected = set(range(min_n, max_n + 1))
        actual = set(nums)
        if actual != expected:
            missing = sorted(expected - actual)
            extra = sorted(actual - expected)
            msg = f"Immediate numeric directory names {sorted(entries)} do not form a consecutive range {min_n}..{max_n}"
            if missing:
                msg += f"; missing {missing}"
            if extra:
                msg += f"; unexpected {extra}"
            raise AssertionError(msg)
        return min_n, max_n
    # assert that it's a directory
    assert os.path.isdir(path), f"{path} is not a directory"

    # assert directory is not empty
    assert any(os.listdir(path)), f"{path} is empty"

    # assert only folders in the first level
    assert all(os.path.isdir(os.path.join(path, subdir)) for subdir in os.listdir(path)), f"Not all items in {path} are directories"

    # assert that all first level subdirectories are nats and ordered
    check_numeric_children_consecutive(path)

    # go into folder titled path/0 and find the set of folder names
    zero_folder = os.path.join(path, "0")
    assert os.path.isdir(zero_folder), f"{zero_folder} is not a directory"

    folder_names = {name for name in os.listdir(zero_folder) if os.path.isdir(os.path.join(zero_folder, name))}

    # assert that set of folder_names is the same across all number directories
    for subdir in os.listdir(path):
        subdir_path = os.path.join(path, subdir)
        if os.path.isdir(subdir_path) and subdir != "0":
            subdir_folder_names = {name for name in os.listdir(subdir_path) if os.path.isdir(os.path.join(subdir_path, name))}
            assert subdir_folder_names == folder_names, f"Folder names in {subdir_path} do not match those in {zero_folder}"

    # assert that all subfolders have the same file names e.g. path/0/subfolder/[1..8].txt matches path/1/
    file_set = None
    for folder in os.listdir(path):
        folder_path = os.path.join(path, folder)
        for subdir in os.listdir(folder_path):
            assert os.path.isdir(os.path.join(folder_path, subdir)), f"{subdir} is not a directory"
            file_names = {name for name in os.listdir(os.path.join(folder_path, subdir)) if os.path.isfile(os.path.join(folder_path, subdir, name))}
            if file_set is None:
                file_set = file_names
            else:
                assert file_set == file_names, f"File names in {subdir} are not consistent with other subdirectories"

# Assumes you have a folder called proofs in the current working directory with the structure enforced by the check_structure function
check_structure("proofs")


In [7]:
!pip install --upgrade huggingface_hub
!pip install --upgrade datasets
import os
os.system("huggingface-cli login --token HF_TOKEN")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [huggingface_hub] [huggingface_hub]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 62.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12/12 [datasets]/12 [datasets]ess]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Traceback (most recent call last):
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/huggingface_hub/utils/_http.py", line 409, in hf_raise_for_status
    response.raise_for_status()
  File "/home/codespace/.local/lib/python3.12/site-packages/requests/models.py", line 1026, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/whoami-v2

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/huggingface_hub/hf_api.py", line 1782, in whoami
    hf_raise_for_status(r)
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/huggingface

256

In [8]:
from datasets import load_dataset
miniF2F_test_df = load_dataset("AI-MO/minif2f_test", split="train").to_pandas().head(40)

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 244/244 [00:00<00:00, 64007.14 examples/s]


In [9]:
import os
import json
from typing import Any, Union

def _read_text_file(path: str) -> str:
    try:
        with open(path, "r", encoding="utf-8") as f:
            return f.read()
    except UnicodeDecodeError:
        with open(path, "r", encoding="latin-1") as f:
            return f.read()

def build_structure(path: str) -> Union[dict[str, Any], str, None]:
    """
    Recursively walk `path`:
      - If it's a directory, return a dict of its children.
      - If it's a .txt file, return its text content.
      - Other files return None (and are skipped by the caller).
    """
    if os.path.isdir(path):
        out: dict[str, Any] = {}
        for entry in sorted(os.listdir(path)):
            full = os.path.join(path, entry)
            if os.path.isdir(full):
                out[entry] = build_structure(full)
            elif os.path.isfile(full) and entry.lower().endswith(".txt"):
                key = os.path.splitext(entry)[0]
                out[key] = _read_text_file(full)
        return out
    elif os.path.isfile(path) and path.lower().endswith(".txt"):
        return _read_text_file(path)
    else:
        return None  # non-txt file or missing; caller can ignore

structure = build_structure("proofs")
structure['0']['AI-MO_Kimina-Prover-Preview-Distill-7B']

{'1': '-- The area of the base of a cone is 30 square units, and its height is 6.5 units. What is the number of cubic units in its volume? Show that it is 65.\n  rw [h₁, h₂, h₃]\n  -- The volume of a cone is given by the formula $V = \\frac{1}{3}Bh$, where $B$ is the area of the base and $h$ is the height.\n  norm_num\n  -- The area of the base of a cone is 30 square units, and its height is 6.5 units. What is the number of cubic units in its volume? Show that it is 65.\n```',
 '2': '-- The area of the base of a cone is 30 square units, and its height is 6.5 units. What is the number of cubic units in its volume? Show that it is 65.\n  calc\n    -- The volume of a cone is given by the formula $V = \\frac{1}{3}Bh$, where $B$ is the area of the base and $h$ is the height.\n    v = 1 / 3 * (b * h) := by rw [h₁]\n    _ = 1 / 3 * (30 * 13 / 2) := by rw [h₂, h₃]\n    _ = 65 := by norm_num\n```',
 '3': "-- The area of the base of a cone is 30 square units, and its height is 6.5 units. What is

In [10]:
miniF2F_test_df.head(5)

,name,informal_prefix,formal_statement
0,mathd_algebra_478,/-- The volume of a cone is given by the formu...,import Mathlib\nimport Aesop\n\nset_option max...
1,numbertheory_4x3m7y3neq2003,/-- Show that there are no integers $x$ and $y...,import Mathlib\nimport Aesop\n\nset_option max...
2,aime_1983_p1,"/-- Let $x$, $y$ and $z$ all exceed $1$ and le...",import Mathlib\nimport Aesop\n\nset_option max...
3,amc12_2001_p5,/-- What is the product of all positive odd in...,import Mathlib\nimport Aesop\n\nset_option max...
4,mathd_algebra_141,/-- A rectangular patio has an area of $180$ s...,import Mathlib\nimport Aesop\n\nset_option max...


In [11]:
row1 = miniF2F_test_df.loc[0]
# 1. Define the proof and context with the Mathlib header

correct_proof_dict = [{
    'formal_statement': row1['formal_statement'],
    'proof': structure['0']['AI-MO_Kimina-Prover-Preview-Distill-7B'][str(i)],
    'project_dir': lean_project_path
} for i in range(1, 9)]

In [12]:
check_proofs_in_parallel(correct_proof_dict)

True

In [13]:
import re

def format_proof(proof: str, model_id: str) -> str:
    """
    Extracts and formats a Lean 4 proof from a larger string based on the model_id.
    """
    match model_id:
        case "stoney0062_Leanabell-Prover-DS-SFT":
            # Use regex to find the content between the lean4 code fences.
            # re.DOTALL allows '.' to match newline characters.
            _match = re.search(r"```lean4(.*?)```", proof, re.DOTALL)

            # If a match is found, return the extracted code, stripped of whitespace.
            if _match:
                return _match.group(1).strip()
            # Return an empty string if the code block isn't found.
            return ""

        case _:
            # For any other model_id, return the original proof unmodified.
            return proof

# print(correct_proof_dict['proof'])
# formatted_proof = format_proof(correct_proof_dict['proof'], "stoney0062_Leanabell-Prover-DS-SFT")
# formatted_proof

In [14]:
print(structure['0']['stoney0062_Leanabell-Prover-DS-SFT']['1'])

Given the volume formula for a cone \( V = \frac{1}{3}Bh \), where \( B \) is the area of the base and \( h \) is the height, we are provided with the base area \( B = 30 \) square units and the height \( h = 6.5 \) units. We need to show that the volume \( V \) is 65 cubic units.
First, substitute the given values into the volume formula:
\[ V = \frac{1}{3} \times 30 \times \frac{13}{2} \]
Next, perform the multiplication:
\[ V = \frac{1}{3} \times 30 \times 6.5 = \frac{1}{3} \times 195 = 65 \]
Thus, the volume \( V \) is indeed 65 cubic units.
</think>
<answer>
```lean4
  -- Substitute the given values into the volume formula
  rw [h₂, h₃] at h₁
  -- Simplify the expression using numerical normalization
  norm_num at h₁
  -- Use linear arithmetic to conclude the proof
  linarith
``` 
</answer>


In [15]:
for num in structure:
    for model in structure[num]:
        if model == "stoney0062_Leanabell-Prover-DS-SFT":
            for proof in structure[num][model]:
                structure[num][model][proof] = format_proof(structure[num][model][proof], model)

In [16]:
print(structure['0']['stoney0062_Leanabell-Prover-DS-SFT']['1'])

-- Substitute the given values into the volume formula
  rw [h₂, h₃] at h₁
  -- Simplify the expression using numerical normalization
  norm_num at h₁
  -- Use linear arithmetic to conclude the proof
  linarith


In [17]:
!pip install bitarray


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [18]:
from bitarray import bitarray
# from google.colab import files

def save_bitarray_to_file(ba: bitarray, filename: str):
    assert isinstance(ba, bitarray), "ba must be a bitarray"
    assert isinstance(filename, str), "filename must be a string"
    with open(filename, 'wb') as f:
        ba.tofile(f)
    # download the file
    # files.download(filename)

def load_bitarray_from_file(filename: str) -> bitarray:
    assert isinstance(filename, str), "filename must be a string"
    ba = bitarray()
    with open(filename, 'rb') as f:
        ba.fromfile(f)
    return ba

In [22]:
import os
import json
from bitarray import bitarray
from typing import Dict
from tqdm.auto import tqdm  # or: from tqdm import tqdm

def _load_checkpoint(path: str, num_problems: int) -> Dict[str, bitarray]:
    if not os.path.exists(path):
        return {}
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    model_dict: Dict[str, bitarray] = {}
    for model, bitstr in data.get("model_dict", {}).items():
        ba = bitarray(bitstr)
        if len(ba) < num_problems:
            ba.extend([False] * (num_problems - len(ba)))
        elif len(ba) > num_problems:
            ba = ba[:num_problems]
        model_dict[model] = ba
    return model_dict

def _save_checkpoint(path: str, model_dict: Dict[str, bitarray]) -> None:
    tmp = f"{path}.tmp"
    serializable = {"model_dict": {m: ba.to01() for m, ba in model_dict.items()}}
    with open(tmp, "w", encoding="utf-8") as f:
        json.dump(serializable, f, ensure_ascii=False)
    os.replace(tmp, path)  # atomic-ish swap

def run_proof_checker_on_all_proofs(
    structure_json: dict,
    df,
    lean_project_path: str,
    checkpoint_path: str,
    max_attempts: int = 8,
    resume: bool = True,
) -> Dict[str, bitarray]:
    """
    Resumable runner with nested progress bars. Assumes structure_json is validated.
    Returns model_dict mapping model name to bitarray of successful problem indices.
    """
    num_problems = len(df)

    # load or initialize
    if resume:
        model_dict = _load_checkpoint(checkpoint_path, num_problems)
    else:
        model_dict = {}

    # ensure all models under "0" are initialized
    for model in structure_json.get("0", {}).keys():
        if model not in model_dict:
            ba = bitarray(num_problems)
            ba.setall(False)
            model_dict[model] = ba

    # outer bar over problems
    for problem_str in tqdm(sorted(structure_json.keys(), key=lambda x: int(x)), desc="problems"):
        if int(problem_str) < 20: # for now bc I paused it mid-way
          continue
        problem_idx = int(problem_str)
        df_row = df.iloc[problem_idx]
        models = list(structure_json[problem_str].items())

        # inner bar over models for this problem
        with tqdm(models, desc=f"problem {problem_idx} models", leave=False, unit="model") as model_bar:
            for model, attempts in model_bar:
                if model_dict[model][problem_idx]:
                    model_bar.set_postfix({"status": "skipped"})
                    print(f"✅ Model {model} succeeded on problem {problem_idx}")
                    continue

                # build proof contexts for all available attempts
                proof_contexts = [
                    {
                        "formal_statement": df_row["formal_statement"],
                        "proof": attempts[str(i)],
                        "project_dir": lean_project_path,
                    }
                    for i in range(1, max_attempts + 1)
                ]
                if not proof_contexts:
                    model_bar.set_postfix({"status": "no attempts"})
                    continue

                success = check_proofs_in_parallel(proof_contexts, parallel_workers=4)
                if success:
                    model_dict[model][problem_idx] = True
                    print(f"✅ Model {model} succeeded on problem {problem_idx}")
                _save_checkpoint(checkpoint_path, model_dict)

    # final persist
    _save_checkpoint(checkpoint_path, model_dict)
    return model_dict

In [ ]:
# !mkdir prover_correctness

In [ ]:
# model_dict = run_proof_checker_on_all_proofs(
#     structure_json=structure,
#     df=miniF2F_test_df,
#     lean_project_path=lean_project_path,
#     checkpoint_path="checkpoint.json",
#     max_attempts=8,
#     resume=True,
# )

problems:   0%|          | 0/40 [00:00<?, ?it/s]

problem 20 models:   0%|          | 0/9 [00:00<?, ?model/s]

An error occurred during proof checking: Command '['lake', 'env', 'lean', '1.lean']' timed out after 60 seconds


problem 21 models:   0%|          | 0/9 [00:00<?, ?model/s]

✅ Model stoney0062_Leanabell-Prover-DS-SFT succeeded on problem 21


...

problem 36 models:   0%|          | 0/9 [00:00<?, ?model/s]

problem 37 models:   0%|          | 0/9 [00:00<?, ?model/s]

An error occurred during proof checking: Command '['lake', 'env', 'lean', '4.lean']' timed out after 60 seconds


problem 38 models:   0%|          | 0/9 [00:00<?, ?model/s]

problem 39 models:   0%|          | 0/9 [00:00<?, ?model/s]

In [ ]:
# import os
# import shutil
# from google.colab import files

# # Save the files
# output_dir = "prover_correctness"
# os.makedirs(output_dir, exist_ok=True)
# for model, ba in model_dict.items():
#     save_bitarray_to_file(ba, os.path.join(output_dir, f"{model}.bin"))

# # Zip the entire directory
# shutil.make_archive("prover_correctness", "zip", output_dir)

# # Download the zip file
# files.download("prover_correctness.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(model_dict.keys())

dict_keys(['AI-MO_Kimina-Prover-Preview-Distill-7B', 'ByteDance-Seed_BFS-Prover', 'Goedel-LM_Goedel-Prover-SFT', 'deepseek-ai_DeepSeek-Prover-V1', 'deepseek-ai_DeepSeek-Prover-V1.5-RL', 'deepseek-ai_DeepSeek-Prover-V2-7B', 'kfdong_STP_model_Lean', 'stoney0062_Leanabell-Prover-DS-SFT', 'wellecks_llmstep-mathlib4-pythia2.8b'])


In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List, Optional, Callable

def check_proofs_in_parallel_output_arr(
    proof_contexts: List[dict],
    parallel_workers: int | None = None,
    on_progress: Optional[Callable[[int], None]] = None,
) -> List[int]:
    """
    Runs multiple proof checks in parallel (threaded).
    Returns a list of 1/0 aligned with the input order.
    Calls on_progress(1) each time an attempt completes.
    """
    assert isinstance(parallel_workers, int) and parallel_workers > 1, "parallel_workers must be > 1"
    if not proof_contexts:
        return []

    results: List[int] = [0] * len(proof_contexts)

    def _run(idx: int, ctx: dict) -> Tuple[int, int]:
        name = f"proof_{idx}"
        ok = check_lean_proof(ctx, name=name)
        return idx, 1 if ok else 0

    with ThreadPoolExecutor(max_workers=parallel_workers) as executor:
        futures = {executor.submit(_run, i, ctx): i for i, ctx in enumerate(proof_contexts, start=1)}
        for future in as_completed(futures):
            idx = futures[future]
            try:
                i, val = future.result()
            except Exception:
                i, val = idx, 0
            results[i - 1] = val
            if on_progress:
                on_progress(1)

    return results

In [ ]:
import os
import json
import numpy as np
from tqdm.auto import tqdm  # progress bar

# If structure_json isn't built yet, do it now:
try:
    structure_json  # type: ignore[name-defined]
except NameError:
    structure_json = build_structure("proofs")

# Robustly find the checkpoint path (use your own if different)
checkpoint_path_candidates = [
    "checkpoint.json",
]

checkpoint_path = next((p for p in checkpoint_path_candidates if os.path.exists(p)), "checkpoint.json")

# Derive problem/model/attempt axes from the verified folder structure
problem_ids = sorted(structure_json.keys(), key=lambda x: int(x))
first_problem = problem_ids[0]
model_names = sorted(structure_json[first_problem].keys())
# assume consistent attempt ids across models/problems (enforced by your check_structure)
attempt_ids = sorted(structure_json[first_problem][model_names[0]].keys(), key=lambda x: int(x))

num_problems = len(problem_ids)
num_models = len(model_names)
num_attempts = len(attempt_ids)

# Load coarse per-problem success bits from checkpoint (pads/truncates to num_problems)
model_bits = _load_checkpoint(checkpoint_path, num_problems)  # Dict[str, bitarray-like]

# Initialize the 3D result tensor (problems × models × attempts) with zeros
results = np.zeros((num_problems, num_models, num_attempts), dtype=np.uint8)

# Choose a parallelism level (> 1 due to helper's assert)
parallel_workers = max(2, (os.cpu_count() or 2))

# Global progress bar over all (problem, model) pairs
with tqdm(total=num_models * num_problems, desc="Evaluating (problem, model) pairs", unit="pair") as pbar:
    for m_idx, model in enumerate(model_names):
        coarse = model_bits.get(model, None)
        if coarse is None:
            # Model missing from checkpoint → count all problems as processed (left zeros)
            pbar.update(num_problems)
            continue

        for p_axis_idx, p_str in enumerate(problem_ids):
            if not bool(coarse[p_axis_idx]):
                # Fast path: checkpoint says no success → keep zeros
                pbar.update(1)
                continue

            # Otherwise, verify each attempt for this (problem, model)
            attempts_dict = structure_json[p_str][model]  # keys like "1","2",...
            formal_statement = df.iloc[p_axis_idx]["formal_statement"]

            # Build proof contexts for the helper
            proof_contexts = []
            for a_str in attempt_ids:
                proof_text = attempts_dict.get(a_str, "")
                proof_contexts.append({
                    "formal_statement": formal_statement,
                    "proof": format_proof(proof_text, model),
                    "project_dir": lean_project_path,
                })

            # Run attempt checks via the threaded helper and fill the slice
            outcomes = check_proofs_in_parallel_output_arr(
                proof_contexts,
                parallel_workers=parallel_workers,
            )
            results[p_axis_idx, m_idx, :] = np.fromiter(outcomes, dtype=np.uint8, count=num_attempts)

            pbar.update(1)

print("results shape:", results.shape)  # (num_problems, num_models, num_attempts)
# Example: results[problem_idx, model_idx, attempt_idx] \in {0,1}

Evaluating (problem, model) pairs:  86%|████████▌ | 310/360 [30:00<05:58,  7.16s/pair]  

An error occurred during proof checking: Command '['lake', 'env', 'lean', 'proof_1.lean']' timed out after 60 seconds
An error occurred during proof checking: Command '['lake', 'env', 'lean', 'proof_5.lean']' timed out after 60 seconds


Evaluating (problem, model) pairs:  87%|████████▋ | 312/360 [31:17<12:07, 15.16s/pair]

An error occurred during proof checking: Command '['lake', 'env', 'lean', 'proof_7.lean']' timed out after 60 seconds


Evaluating (problem, model) pairs:  92%|█████████▏| 331/360 [34:02<02:40,  5.53s/pair]

An error occurred during proof checking: Command '['lake', 'env', 'lean', 'proof_5.lean']' timed out after 60 secondsAn error occurred during proof checking: Command '['lake', 'env', 'lean', 'proof_6.lean']' timed out after 60 seconds

An error occurred during proof checking: Command '['lake', 'env', 'lean', 'proof_7.lean']' timed out after 60 seconds
An error occurred during proof checking: Command '['lake', 'env', 'lean', 'proof_8.lean']' timed out after 60 seconds


Evaluating (problem, model) pairs: 100%|██████████| 360/360 [35:16<00:00,  5.88s/pair]

results shape: (40, 9, 8)


In [48]:
import json
from typing import List

def results_to_nested_dict(
    results,                      # numpy array, shape: (num_problems, num_models, num_attempts)
    model_names: List[str],
    problem_ids: List[str],
) -> Dict[str, Dict[str, list]]:
    """Return {model_id: {problem_id: [0/1, ...]}}."""
    num_problems, num_models, num_attempts = results.shape
    assert len(model_names) == num_models
    assert len(problem_ids) == num_problems

    payload: Dict[str, Dict[str, list]] = {}
    for m_idx, model in enumerate(model_names):
        per_model: Dict[str, list] = {}
        for p_idx, prob in enumerate(problem_ids):
            per_model[str(prob)] = results[p_idx, m_idx, :].astype(int).tolist()
        payload[str(model)] = per_model
    return payload

def save_results_nested_json(
    results,
    model_names: List[str],
    problem_ids: List[str],
    out_path: str = "proof_outcomes_by_model.json",
) -> str:
    payload = results_to_nested_dict(results, model_names, problem_ids)
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(payload, f, indent=2, ensure_ascii=False)
    return out_path

# Example:
# save_results_nested_json(results, model_names, problem_ids, "proof_outcomes_by_model.json")


In [52]:
save_results_nested_json(results, model_names, problem_ids, "proof_outcomes_by_model.json")

'proof_outcomes_by_model.json'

In [50]:
def results_to_problem_nested_dict(
    results,                      # numpy array, shape: (num_problems, num_models, num_attempts)
    model_names: List[str],
    problem_ids: List[str],
) -> Dict[str, Dict[str, list]]:
    """
    Return a dict nested by problem, then model:

    {
      "<problem_id>": {
        "<model_id>": [0/1, 0/1, ...],
        ...
      },
      ...
    }
    """
    num_problems, num_models, _ = results.shape
    assert len(model_names) == num_models
    assert len(problem_ids) == num_problems

    payload: Dict[str, Dict[str, list]] = {}
    for p_idx, prob in enumerate(problem_ids):
        per_problem: Dict[str, list] = {}
        for m_idx, model in enumerate(model_names):
            per_problem[str(model)] = results[p_idx, m_idx, :].astype(int).tolist()
        payload[str(prob)] = per_problem
    return payload


def save_results_nested_by_problem_json(
    results,
    model_names: List[str],
    problem_ids: List[str],
    out_path: str = "proof_outcomes_by_problem.json",
) -> str:
    """
    Writes JSON nested by problem → model → outcomes.
    """
    payload = results_to_problem_nested_dict(results, model_names, problem_ids)
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(payload, f, indent=2, ensure_ascii=False)
    return out_path


In [53]:
save_results_nested_by_problem_json(results, model_names, problem_ids, "proof_outcomes_by_problem.json")


'proof_outcomes_by_problem.json'